In [12]:
import os
import pandas as pd
import requests
from PyPDF2 import PdfReader
from io import BytesIO

In [13]:
# Load your DataFrame (assuming it has 'date', 'year', and 'link' columns)
df = pd.read_csv('imf_statements_links.csv')
# Remove repeated 'https://meetings.imf.org' in Link column
df['Link'] = df['Link'].str.replace(r'(https://meetings.imf.org)+', 'https://meetings.imf.org', regex=True)

In [15]:
import requests
import os
from io import BytesIO
from PyPDF2 import PdfReader
import pandas as pd

# Function to download PDF
def download_pdf(link):
    try:
        response = requests.get(link)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx and 5xx)
        return response.content
    except requests.RequestException as e:
        print(f"Error downloading {link}: {e}")
        return None

# Function to extract text from PDF content
def extract_text_from_pdf(content):
    try:
        pdf_bytes = BytesIO(content)
        pdf_reader = PdfReader(pdf_bytes)
        text = ''
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        print(f"Error reading PDF content: {e}")
        return None

# Function to save PDF to a specified directory
def save_pdf(content, year, title, data_directory="data"):
    directory = os.path.join(data_directory, str(year))
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, f"{title}.pdf")
    with open(file_path, 'wb') as f:
        f.write(content)
    return file_path

# Function to download PDFs and store file paths
def download_files(df):
    file_paths = []
    for _, row in df.iterrows():
        content = download_pdf(row['Link'])
        if content:
            file_path = save_pdf(content, row['Year'], row['Title'])
            file_paths.append(file_path)
        else:
            file_paths.append(None)  # Append None if download failed
    df['File_Path'] = file_paths
    return df

# Function to read and extract text from downloaded PDF files
def read_files(df):
    texts = []
    for _, row in df.iterrows():
        if row['File_Path']:
            with open(row['File_Path'], 'rb') as f:
                content = f.read()
            text = extract_text_from_pdf(content)
            texts.append(text)
        else:
            texts.append(None)  # Append None if file is not available
    df['Extracted_Text'] = texts
    return df

# Example of usage with your DataFrame
def process_dataframe(df):
    # Step 1: Download files
    df = download_files(df)
    
    # Step 2: Read files and extract text
    df = read_files(df)
    
    return df

# Assuming df is your input DataFrame
processed_df = process_dataframe(df)

# Save the updated dataframe
processed_df.to_csv("processed_data.csv", index=False)

print(processed_df)


Error downloading https://meetings.imf.org/en/sitecore/media-library/AMSM/Files/AM2021/IMFC/India: 404 Client Error: Not Found for url: https://meetings.imf.org/external/error.htm
Error downloading https://meetings.imf.orghttps://www.imf.org/~/media/AMSM/Files/AM2020/IMFC/cog.pdf?la=en: HTTPSConnectionPool(host='meetings.imf.orghttps', port=443): Max retries exceeded with url: /www.imf.org/~/media/AMSM/Files/AM2020/IMFC/cog.pdf?la=en (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000222FD0D8050>: Failed to resolve 'meetings.imf.orghttps' ([Errno 11001] getaddrinfo failed)"))
Error downloading https://meetings.imf.orghttps://www.imf.org/~/media/AMSM/Files/AM2020/IMFC/cogf.pdf?la=en: HTTPSConnectionPool(host='meetings.imf.orghttps', port=443): Max retries exceeded with url: /www.imf.org/~/media/AMSM/Files/AM2020/IMFC/cogf.pdf?la=en (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000222FD0D8550>: Failed to resolve 'meeti

FileNotFoundError: [Errno 2] No such file or directory: 'data\\2016\\IMFC Statement by Rosine Coulibaly/Sori, Minister of Economy, Finance and Development, Burkina Faso.pdf'